In [1]:
from IPython.display import HTML
#DOESN'T SEEM T WORK
HTML("""
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
""")
#aim is to autorun all

In [2]:
from Jumpscale import j
simulation = j.tools.tfsimulation.simulation_get()
environment = simulation.environment_get("main")
from IPython.display import Markdown as md
import numpy as np
import bqplot.pyplot as plt
from bqplot import *
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import HBox, Label, Layout


## input arguments

In [3]:
environment.cost_power_kwh = "0.15 USD"
environment.cost_rack_unit = "12 USD"
environment.bandwidth_mbit = 20

# sales arguments
environment.sales_price_cu = "8 USD"
environment.sales_price_su = "5 USD"

device_edge = simulation.device_get("edge1", environment=environment)
# print(d1)

switch = simulation.device_get("switch", environment=environment)

environment.device_add("edge1", device_edge, 20)
environment.device_add("switch", switch, 2)

# print(environment)

simulation_run = simulation.run()

# month:growth_percent of nodes being added
simulation_run.growth_percent_set("3:5,11:8,24:10,36:12,48:5,60:5")

# means at end of period we produce 40% more cpr (*1.4)
# cpr = capacity production rate (is like hashrate of bitcoin)
simulation_run.cpr_improve_set("71:40")

# price of a capacity unit goes down over time, here we say it will go down 40%
# means we expect price to be lowered by X e.g. 40 (*0.6)
simulation_run.cpr_sales_price_decline_set("0:100,71:40")

# utilization of the nodes, strating with 0
simulation_run.utilization_set("30:80,71:90")

# super important factor, how does token price goes up, this is ofcourse complete speculation, no-one knows
simulation_run.tokenprice_set("0:0.15,30:2,71:2.5")

# are the parameters for a batch of nodes to be added
nb = simulation_run.nodes_batch_template_get(environment)
# print(nb)

# do the calculation of the simulation
simulation_run.calc(nr_start_nodes=1500, months_remaining_start_nodes=36,environment=environment)


In [4]:
C="""
```python
%s
```
"""%In[2]
m=md(C)
m


```python
from Jumpscale import j
simulation = j.tools.tfsimulation.simulation_get()
environment = simulation.environment_get("main")
from IPython.display import Markdown as md
import numpy as np
import bqplot.pyplot as plt
from bqplot import *
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import HBox, Label, Layout
```


### calculated edge device

In [5]:
device_edge

# JSXOBJ:
## threefold.simulation.device
ID: 6
 - name                : edge1
 - description         : 
 - device_template_name: edge1
 - cost                : 3,627
 - power               : 222
 - rackspace_u         : 2
 - cru                 : 16
 - sru                 : 1000
 - hru                 : 60000
 - mru                 : 128
 - su                  : 58
 - cu                  : 31
 - cu_passmark         : 771
 - su_perc             : 62.95%
 - cu_perc             : 37.05%
 - cpr                 : 105
 - cost_su_capex       : 39.14204775394577
 - cost_cu_capex       : 42.321697039994646
 - cost_power          : 23.976
 - cost_rack           : 24
 - cost_month          : 108.426
 - cost_cu_month       : 1.2651702021666555
 - cost_su_month       : 1.170117361392149


### environment of 1500 edge devices

Is a complete environment with switches and 1500 edge devices


In [6]:
environment

# JSXOBJ:
## threefold.simulation.environment
ID: 5
 - name                : main
 - description         : 
 - cost_power_kwh      : 0.15
 - cost_rack_unit      : 12
 - cost                : 81,540
 - power               : 4,640
 - rackspace_u         : 42
 - cru                 : 320
 - sru                 : 20000
 - hru                 : 1200000
 - mru                 : 2560
 - su                  : 1166
 - cu                  : 634
 - cu_passmark         : 771
 - su_perc             : 62.95%
 - cu_perc             : 37.05%
 - cpr                 : 2,118
 - cost_su_capex       : 43.998381222177244
 - cost_cu_capex       : 47.610016139942005
 - cost_power          : 501.12
 - cost_rack           : 504
 - cost_month          : 2364.12
 - cost_cu_month       : 1.380375169938186
 - cost_su_month       : 1.275661675435046
 - bandwidth_mbit      : 20
 - cost_bandwidth      : 0
 - nr_devices          : 20
 - sales_price_cu      : 8
 - sales_price_su      : 5
 - sales_price_nu      : 0
 - sa

### Growth Percent per month

Choose Number of months



In [7]:
def plotGrowth(nrmonths):
    plt.figure(title='Growth Percent Per Month (%s Months)' % nrmonths)
    plt.plot(np.arange(nrmonths), simulation_run.sheet.rows["growth_percent"].cells[:nrmonths])
    plt.show()
    
interact(plotGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))


<function __main__.plotGrowth(nrmonths)>

### Number of nodes

Choose Number of months



In [8]:
def plotNodeGrowth(nrmonths):
    plt.figure(title='Nr Nodes (%s Months)' % nrmonths)
    plt.plot(np.arange(nrmonths), simulation_run.sheet.rows["nrnodes"].cells[:nrmonths])
    plt.show()

In [50]:
interact(plotNodeGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

interactive(children=(IntSlider(value=72, description='Nr months', max=72, min=24, step=24), Output()), _dom_c…

<function __main__.plotNodeGrowth(nrmonths)>

### Number of racks per month

Choose Number of months


In [49]:
def plotRackGrowth(nrmonths):
    plt.figure(title='Nr Racks Occupied. (%s Months)' % nrmonths)
    plt.plot(np.arange(nrmonths), simulation_run.sheet.rows["nrracks"].cells[:nrmonths])
    plt.show()


In [51]:
interact(plotRackGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

interactive(children=(IntSlider(value=72, description='Nr months', max=72, min=24, step=24), Output()), _dom_c…

<function __main__.plotRackGrowth(nrmonths)>

### Revenue per month

Choose Number of months


In [46]:
def revenuGrowth(nrmonths):
    plt.figure(title='Revenue Per Month (kUSD). (%s Months)'% nrmonths)
    plt.plot(np.arange(nrmonths), [int(i/1000) for i in simulation_run.sheet.rows["revenue_month"].cells[:nrmonths]])
    plt.show()

In [47]:
interact(revenuGrowth, nrmonths=widgets.IntSlider(min=24, max=72, step=24, value=72, description='Nr months'))

interactive(children=(IntSlider(value=72, description='Nr months', max=72, min=24, step=24), Output()), _dom_c…

<function __main__.revenuGrowth(nrmonths)>

In [11]:
plt.figure(title='Revenue Per Year (million USD).')
plt.bar(np.arange(0,6), [int(i/1000) for i in simulation_run.sheet.rows["revenue_month"].aggregate("Y")])
plt.show()

In [12]:
plt.figure(title='Recurring Revenue Over 60 Months (million USD).')
plt.bar(np.arange(simulation_run.sheet.nrcols), [round(float(i)*60/1000,2) for i in simulation_run.sheet.rows["revenue_month"].cells])
plt.show() 

In [13]:
simulation_run


 - growth_percent       0, 5.0, 6.12, 7.25, 8.15, 8.62, 9.08, 9.54, 10.0, 10.5, 11.0, 11.5, 12.0, 10.25, 8.5, 6.75, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0
 - cpr_improve          0, 1.69, 3.38, 5.07, 6.76, 8.45, 10.14, 11.83, 13.52, 15.21, 16.9, 18.59, 20.28, 21.97, 23.66, 25.35, 27.04, 28.73, 30.42, 32.11, 33.8, 35.49, 37.18, 38.87
 - cpr_sales_price_decline 100, 97, 95, 92, 90, 87, 85, 82, 80, 77, 75, 72, 70, 67, 65, 62, 59, 57, 54, 52, 49, 47, 44, 42
 - utilization          0, 8.0, 16.0, 24.0, 32.0, 40.0, 48.0, 56.0, 64.0, 72.0, 80.0, 80.73, 81.46, 82.2, 82.93, 83.66, 84.39, 85.12, 85.85, 86.59, 87.32, 88.05, 88.78, 89.51
 - tokenprice           0.15, 0.33, 0.52, 0.71, 0.89, 1.07, 1.26, 1.44, 1.63, 1.81, 2.0, 2.04, 2.07, 2.11, 2.15, 2.18, 2.22, 2.26, 2.29, 2.33, 2.37, 2.4, 2.44, 2.48
 - nrnodes_new          1500, 79, 113, 161, 226, 303, 411, 562, 778, 1092, 1550, 2225, 3233, 3820, 4177, 4169, 3695, 4278, 4952, 5733, 6636, 7682, 8893, 10295
 - nrnodes              1500, 1653, 1953, 2

In [14]:
plt.figure(title='Token Price (TFT).')
plt.plot(np.arange(simulation_run.sheet.nrcols), [round(i,2) for i in simulation_run.sheet.rows["tokenprice"].cells])
plt.show()